## 도구(Tools)가 결합된 비주얼 에이전트

In [ ]:
%pip install -U ddgs

In [ ]:
from dotenv import load_dotenv
load_dotenv()

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
# Langchain에서 제공하는 무료 검색 툴
from langchain_community.tools import DuckDuckGoSearchRun 

# 검색 도구 인스턴스 생성
search = DuckDuckGoSearchRun()

## 에이전트 로직 설계 (함수 분리)

In [ ]:
# 장소 식별 함수
def identify_landmark(img_url):
    """ 1단계 : 이미지 식별 - 사진을 보고 장소 이름을 반환 """

    model = ChatOpenAI(model='gpt-4o', temperature=0)

    prompt = "이 사진속 장소의 정확한 이름만 알려줘. 다른말은 하지 말고"

    message = HumanMessage(
        content = [
            {'type':'text', 'text' : prompt},
            {'type':'image_url', 'image_url':{"url":img_url}}
        ]
    )

    r = model.invoke([message])
    return r.content 

In [ ]:
# 정보 검색 함수
def get_travel_info(keyword):
    """ 2단계 : 정보 검색 - 키워드로 인터넷 검색 수행 """
    print(f"검색어 : {keyword}")

    search_query = f"{keyword} travel guide insteresting facts entry fee"
    
    result = search.invoke(search_query)    # search = DuckDuckGoSearchRun()
    return result

## 에이전트 통합

In [ ]:
# 입력(이미지) 들어오면 -> 식별 -> 검색 -> 가이드 생성

def visual_agent(img_url):
    """ 통합 에이전트 함수 """
    
    print("이미지 분석중 ...")
    landmark_name = identify_landmark(img_url)
    print(f"식별된 장소 : {landmark_name}")

    print("정보 검색중 ...")
    info = get_travel_info(landmark_name)

    final_model = ChatOpenAI(model='gpt-5-nano', temperature=0.8)

    prompt = f"""
    다음 정보를 바탕으로 '{landmark_name}'에 대한 친절한 여행 가이드 멘트를 한글로 작성해줘 사용하는 언어도 포함시켜서 같이 알려줘 : 
    
    [정보]
    {info}
    """
    
    summary = final_model.invoke(prompt)
    return summary.content

In [ ]:
image = "https://res.klook.com/image/upload/w_750,h_469,c_fill,q_85/w_80,x_15,y_15,g_south_west,l_Klook_water_br_trans_yhcmh3/activities/rog5yjqki6dgbhe9811q.jpg"

result = visual_agent(image)

print(result)